In [1]:
import pymongo

In [2]:
client = pymongo.MongoClient("mongodb://15.165.36.113:27017/")
client

MongoClient(host=['15.165.36.113:27017'], document_class=dict, tz_aware=False, connect=True)

In [3]:
musinsa = client.crawling.musinsa
musinsa

Collection(Database(MongoClient(host=['15.165.36.113:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'musinsa')

In [4]:
%%writefile musinsa.py

import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

all_page_products = []

def get_total_page():
    url = "https://search.musinsa.com/ranking/best?u_cat_cd=002"
    html_page = requests.get(url).content
    soup = BeautifulSoup(html_page, 'html.parser')
    total_page = soup.select_one("#goodsRankForm > div.right_contents.hover_box > div.boxed-list-wrapper.rank-shop > div.thumbType_box.box > span.pagingNumber > span.totalPagingNum").text
    total_page = int(total_page)
    return total_page

def get_musinsa_product(category="002"):
    total_page = get_total_page()
    for page in tqdm_notebook(range(1, total_page+1)):
        url = "https://search.musinsa.com/ranking/best?mainCategory=002&page={}".format(page)
        html_page = requests.get(url.format(page)).content
        soup = BeautifulSoup(html_page, 'html.parser')
        products = soup.select("#goodsRankList > li")
        for product in products:
            # 상품 id
            product_id = product['data-goods-no']
            img_url = product.select_one("img.lazyload").get("data-original")
            product_brand = product.select_one("p.item_title").text
            product_detail_url = product.select_one("p.list_info > a").get("href")
            product_name = product.select_one("p.list_info").text.strip()
            product_price = product.select_one("p.price").text.split()
            product_price = float(product_price[-1].replace(",", "").replace("원", ""))
            start_count_elem = product.select_one("p.point > span.count")
            if start_count_elem:
                star_count = int(start_count_elem.text.replace(",", ""))
            else:
                star_count = 0
            item = {
                "product_id": product_id,
                "img_url": img_url,
                "product_brand":product_brand.strip(),
                "product_detail_url":product_detail_url,
                "product_name":product_name.strip(),
                "product_price":product_price,
                "star_count":star_count
            }
            all_page_products.append(item)
    return all_page_products

Overwriting musinsa.py


In [5]:
import musinsa as ms

In [6]:
items = ms.get_musinsa_product()
len(items)

  0%|          | 0/100 [00:00<?, ?it/s]

9000

In [7]:
items

[{'product_id': '2081554',
  'img_url': 'https://image.msscdn.net/images/goods_img/20210823/2081554/2081554_2_125.jpg',
  'product_brand': '노스페이스',
  'product_detail_url': 'https://store.musinsa.com/app/goods/2081554?loc=goods_rank',
  'product_name': 'NJ1DM62A 남성 1996 에코 눕시 자켓',
  'product_price': 299000.0,
  'star_count': 1932},
 {'product_id': '865862',
  'img_url': 'https://image.msscdn.net/images/goods_img/20180921/865862/865862_3_125.jpg',
  'product_brand': '무신사 스탠다드',
  'product_detail_url': 'https://store.musinsa.com/app/goods/865862?loc=goods_rank',
  'product_name': '캐시미어 블렌드 오버사이즈 싱글 코트 [블랙]',
  'product_price': 139900.0,
  'star_count': 16135},
 {'product_id': '2014490',
  'img_url': 'https://image.msscdn.net/images/goods_img/20210630/2014490/2014490_8_125.jpg',
  'product_brand': '후아유',
  'product_detail_url': 'https://store.musinsa.com/app/goods/2014490?loc=goods_rank',
  'product_name': '리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U',
  'product_price': 79920.0,
  'star_count': 1642}

In [8]:
musinsa.insert_many(items)

In [9]:
import pandas as pd
df_musinsa = pd.DataFrame(items)
df_musinsa.columns

Index(['product_id', 'img_url', 'product_brand', 'product_detail_url',
       'product_name', 'product_price', 'star_count', '_id'],
      dtype='object')

무신사 outer 베스트 상품 메인 크롤링

크롤링 결과(상품 id, 브랜드, 상품명, 가격, 별점 개수)를 mongodb에 넣기

가격이 평균 이하인 데이터만 mongodb에서 추출하여 DataFrame으로 표현

In [39]:
column_names = ['product_id', 'product_brand', 'product_name', 'product_price', 'star_count']
df_musinsa = df_musinsa[column_names]
df_musinsa.head()

,product_id,product_brand,product_name,product_price,star_count
0,2081554,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1932
1,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
2,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
3,1575618,쿠어,MTR 발마칸 코트 (딥브라운),279650.0,422
4,2105483,노스페이스,NC1DM73A 고 프리 하이브리드 다운 코트 EX,269100.0,285


In [13]:
musinsa2 =client.crawling.musinsa2

In [14]:
insert_items = df_musinsa.to_dict('records')
insert_items

[{'product_id': '2081554',
  'product_brand': '노스페이스',
  'product_name': 'NJ1DM62A 남성 1996 에코 눕시 자켓',
  'product_price': 299000.0,
  'star_count': 1932},
 {'product_id': '865862',
  'product_brand': '무신사 스탠다드',
  'product_name': '캐시미어 블렌드 오버사이즈 싱글 코트 [블랙]',
  'product_price': 139900.0,
  'star_count': 16135},
 {'product_id': '2014490',
  'product_brand': '후아유',
  'product_name': '리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U',
  'product_price': 79920.0,
  'star_count': 1642},
 {'product_id': '1575618',
  'product_brand': '쿠어',
  'product_name': 'MTR 발마칸 코트 (딥브라운)',
  'product_price': 279650.0,
  'star_count': 422},
 {'product_id': '2105483',
  'product_brand': '노스페이스',
  'product_name': 'NC1DM73A 고 프리 하이브리드 다운 코트 EX',
  'product_price': 269100.0,
  'star_count': 285},
 {'product_id': '2059078',
  'product_brand': '케이투',
  'product_name': '씬에어(THINAIR) ECO LITE 다운 자켓 미드나잇 블랙',
  'product_price': 295000.0,
  'star_count': 872},
 {'product_id': '2108965',
  'product_brand': '디아프바인',
  'product_name': 

In [15]:
musinsa2.insert_many(insert_items)

In [37]:
results = musinsa2.find()
results

In [38]:
pd.DataFrame(results)

,_id,product_id,product_brand,product_name,product_price,star_count
0,61b043e237d9fd6f690d812a,2081554,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1932
1,61b043e237d9fd6f690d812b,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
2,61b043e237d9fd6f690d812c,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
3,61b043e237d9fd6f690d812d,1575618,쿠어,MTR 발마칸 코트 (딥브라운),279650.0,422
4,61b043e237d9fd6f690d812e,2105483,노스페이스,NC1DM73A 고 프리 하이브리드 다운 코트 EX,269100.0,285
...,...,...,...,...,...,...
8995,61b043e237d9fd6f690da44d,2145349,비바스튜디오,REVERSIBLE FLEECE JACKET KA [PURPLE],138600.0,1
8996,61b043e237d9fd6f690da44e,2247353,카고브로스,CB 셰르파 자켓 (블랙),95000.0,0
8997,61b043e237d9fd6f690da44f,2245176,라티젠,LO BOLERO & BUSTIER KNIT SET(BLACK),88200.0,0
8998,61b043e237d9fd6f690da450,2244337,플로피엔디스크,헤링본 오버코트,289000.0,0


In [42]:
df_musinsa

,product_id,product_brand,product_name,product_price,star_count
0,2081554,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1932
1,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
2,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
3,1575618,쿠어,MTR 발마칸 코트 (딥브라운),279650.0,422
4,2105483,노스페이스,NC1DM73A 고 프리 하이브리드 다운 코트 EX,269100.0,285
...,...,...,...,...,...
8995,2145349,비바스튜디오,REVERSIBLE FLEECE JACKET KA [PURPLE],138600.0,1
8996,2247353,카고브로스,CB 셰르파 자켓 (블랙),95000.0,0
8997,2245176,라티젠,LO BOLERO & BUSTIER KNIT SET(BLACK),88200.0,0
8998,2244337,플로피엔디스크,헤링본 오버코트,289000.0,0


In [44]:
price_avg = df_musinsa.product_price.mean()
price_avg

195805.81466666667

In [47]:
QUERY = { "product_price" : {"$lte" : price_avg}}
results = musinsa2.find(QUERY)
pd.DataFrame(results)

,_id,product_id,product_brand,product_name,product_price,star_count
0,61b043e237d9fd6f690d812b,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
1,61b043e237d9fd6f690d812c,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
2,61b043e237d9fd6f690d8131,2037171,스파오,[허니푸퍼] 리버서블 푸퍼_SPJDB4VC14,69900.0,4207
3,61b043e237d9fd6f690d8132,1535337,코드그라피,[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_쿨그레이,124000.0,4317
4,61b043e237d9fd6f690d8134,1611891,라퍼지 포 우먼,울리치 트렌치 맥코트_Black,108000.0,2693
...,...,...,...,...,...,...
5839,61b043e237d9fd6f690da44a,2127420,엘엠씨,LMC FLEECE TEAM SETUP cream,158000.0,3
5840,61b043e237d9fd6f690da44d,2145349,비바스튜디오,REVERSIBLE FLEECE JACKET KA [PURPLE],138600.0,1
5841,61b043e237d9fd6f690da44e,2247353,카고브로스,CB 셰르파 자켓 (블랙),95000.0,0
5842,61b043e237d9fd6f690da44f,2245176,라티젠,LO BOLERO & BUSTIER KNIT SET(BLACK),88200.0,0
